# FA7 - AES 128 DFA on real target

With this exercise, you will practice fault injection on a real target doing AES 128 encryption.

# Challenge

Retrieve the AES key used to encrypt the data

### Characterization

To characterize the best parameters, you will have to glitch a simple function which counts up to 2500. If the right parameters are set, the device should reply with a value different than that.

### Attack

To attack this implementation, you have several options :

 - Attacks on the 8th round of AES
 - Attacks on the 9th round of AES (See the slides about fault propagation to see the patterns)
 - Attacks on the 10th round, before the SBox operation
 - Attacks on the 10th round, during the last AddRoundKey

Some of the attacks are implemented in the [phoenixAES](https://github.com/SideChannelMarvels/JeanGrey) library

In [ ]:
#Imports

import chipwhisperer as cw
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
# Chipwhisperer basic configuration

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)
prog = cw.programmers.STM32FProgrammer
scope.default_setup()

#Chipwhisperer clock is fed to the target chip
scope.clock.clkgen_freq = 24E6
target.baud = 230400*24/7.37

cw.set_all_log_levels(cw.logging.CRITICAL)

# Already decimate the ADC samples. This way, ext_offset matches the ADC samples
scope.adc.samples = 8000
scope.adc.decimate = 4

In [ ]:
#Helper functions

def target_reset():
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)
    target.flush()


In [ ]:
#Test that everything goes well by sending the loop command and check its output

target_reset()
scope.arm()
target.simpleserial_write("g", bytearray([]))
scope.capture()
val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)
# Should print
# {'valid': True, 'payload': CWbytearray(b'c4 09 00 00'), 'full_response': CWbytearray(b'00 72 04 c4 09 00 00 15 00'), 'rv': bytearray(b'\x00')}

In [ ]:
#Encrypt a plaintext and retrieve the ciphertext
# This can be used as your reference ciphertext for further attacks

target_reset()
scope.arm()
target.simpleserial_write("p", b'0123456789abcdef')
scope.capture()
val = target.simpleserial_read_witherrors('r', 16, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    print(response)
    CLEAN_CIPHERTEXT = response


In [ ]:
# For the last encryption, show the associated power trace

plt.plot(scope.get_last_trace())

In [ ]:
#Basic glitch settings

scope.glitch.clk_src = "clkgen" # glitch counters use the main clock (24MHz)
scope.glitch.output = "glitch_only" # Glitch output signal
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.io.glitch_lp = True # Enable low power MOSFET
scope.io.glitch_hp = True # Enable high power MOSFET
scope.glitch.repeat = 7 # Number of successive glitches, 7 should be OK for our target

In [ ]:
# Characterization loop

# Thic block will iterate over some of the glitch parameters to locate the "sweet spot"
# where the glitches produce an effect

# Change this block accordingly in order to find the parameters that produce
# the maximum amount of glitches before going further the template

# ext_offset is the number of clock cycles after the trigger

for ext_offset in range(1,2):
    scope.glitch.ext_offset = ext_offset
    
    # offset is the percentage into the clock cycle where the glitch starts
    for offset in np.arange(-40.0, 40.0, 0.2):
        scope.glitch.offset = offset
        
        # Width is the percentage of the period to glitch
        for width in  np.arange(32, 39, 0.2):
            scope.glitch.width = width
            
            # Repeat each iteration 5 times to ensure we get consistant results
            count = 0
            for _ in range(5):
                
                #Fun starts here
                target_reset()
                scope.arm()
                target.simpleserial_write("g", bytearray([]))
                ret = scope.capture()
                
                # read response from target
                val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=30)
                
                # If no response from the chip, assume it crashed
                if val['valid'] is False:
                    print("X", end="")
                    continue
                
                # if counter is different thant the correct value, print the results
                counter = int.from_bytes(val['payload'], byteorder='little')
                if counter != 2500:
                    count += 1
                    print(f"*{counter:08d}*", end="")
                    
                else:
                    print(".", end="")
            if count > 0:
                print()
                print(f"Count {count:02d}\text_offset {ext_offset:02d}\tWidth {width:0.2f}\tOffset {offset:0.2f}")

In [ ]:
# ext_offset is the number of clock cycles after the trigger
for ext_offset in range(1,30): #TODO set ext_offset to reach the desired round
    scope.glitch.ext_offset = ext_offset
    
    # offset is the percentage into the clock cycle where the glitch starts
    for offset in np.arange(-30.0, -20.0, 0.2): # TODO shorten range
        scope.glitch.offset = offset
        
        # Width is the percentage of the period to glitch
        for width in  np.arange(38, 39, 0.2): # TODO shorten range
            scope.glitch.width = width


            for _ in range(5):

                #Fun starts here

                target_reset()
                scope.arm()
                # Send dummy password
                target.simpleserial_write("p", b'0123456789abcdef')
                ret = scope.capture()

                val = target.simpleserial_read_witherrors('r', 16, glitch_timeout=10, timeout=30)
                if val['valid'] is False:
                    print("X", end="")
                    continue

                result = val['payload']
                if result != CLEAN_CIPHERTEXT:
                    xor = bytes(a ^ b for (a, b) in zip(CLEAN_CIPHERTEXT, result))
                    print()
                    print(f"ext_offset {ext_offset:02d}\tWidth {width:0.2f}\tOffset {offset:0.2f}\tResult {result.hex()}\txor: {xor.hex()}")
                    try:
                        print(result.decode())
                    except:
                        pass

                else:
                    print(".", end="")


In [ ]:
scope.dis()